# Rossman Sales Prediction

**This analysis seeks to use linear regression to predict sales at Rossman stores across Germany.**

Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance.  (Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality.)  With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.  Reliable sales forecasts enable store managers to create effective staff schedules that increase productivity and motivation.  By helping Rossmann create a robust prediction model, we can help store managers stay focused on what’s most important to them: their customers and their teams!

In [1]:
!pip install ipython-sql


In [4]:
%load_ext sql	

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [0]:
# Load the appropriate Python libraries.
import pandas as pd
import seaborn as sns
sns.set(style="darkgrid")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [5]:
%sql sqlite://

'Connected: @None'

In [6]:
%%sql   
CREATE TABLE EMPLOYEE(firstname varchar(50),lastname varchar(50));  
INSERT INTO EMPLOYEE VALUES('Tom','Mitchell');  
INSERT INTO EMPLOYEE VALUES('Jack','Ryan');

 * sqlite://
Done.
1 rows affected.
1 rows affected.


[]

In [11]:
%sql SELECT * from EMPLOYEE where firstname = "Tom";  

 * sqlite://
Done.


firstname,lastname
Tom,Mitchell


In [9]:
%sql select * from Crops

 * sqlite://
(sqlite3.OperationalError) no such table: Crops
[SQL: select * from Crops]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [0]:
# Read the CSV file with training data.
url = 'http://bit.ly/rossman_train'
df_train = pd.read_csv(url)

### DATA EXPLORATION

To begin, we will explore the structure of the training data and resolve missing values, if any.

In [0]:
# Show the dimensions (rows and columns) of the training data.
df_train.shape

(508604, 10)

In [0]:
# Preview the first few records in the training data.
df_train.head(10)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StoreType,Assortment,CompetitionDistance
0,539,5,2013-07-05,11498,1168,1,1,a,a,770.0
1,720,1,2013-05-13,8097,822,1,1,a,c,15320.0
2,936,7,2014-03-16,0,0,0,0,a,a,580.0
3,157,6,2013-11-09,6906,768,1,0,a,c,2950.0
4,665,1,2013-07-22,6505,1206,1,0,a,a,90.0
5,623,2,2014-11-18,6054,757,1,0,a,a,4080.0
6,225,1,2014-01-27,3936,475,1,0,d,a,10180.0
7,494,1,2014-12-29,8281,1037,1,0,b,a,1260.0
8,317,3,2013-11-20,4905,507,1,1,d,a,3140.0
9,477,4,2014-01-16,5098,405,1,0,d,a,770.0


In [0]:
# List all columns and their data types.  (Use .columns or .dtypes.)
df_train.dtypes

Store                    int64
DayOfWeek                int64
Date                    object
Sales                    int64
Customers                int64
Open                     int64
Promo                    int64
StoreType               object
Assortment              object
CompetitionDistance    float64
dtype: object

In [0]:
# Display summary stats for the numeric columns in the data frame.
df_train.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,CompetitionDistance
count,508604.000000,508604.000000,508604.000000,508604.000000,508604.000000,508604.000000,507301.000000
mean,558.596423,3.998142,5769.155258,632.638469,0.829899,0.381240,5429.155708
std,321.803180,1.996213,3850.229142,464.498889,0.375722,0.485692,7705.835618
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,20.000000
25%,281.000000,2.000000,3722.000000,404.000000,1.000000,0.000000,710.000000
50%,558.000000,4.000000,5742.000000,609.000000,1.000000,0.000000,2330.000000
75%,838.000000,6.000000,7848.000000,836.000000,1.000000,1.000000,6890.000000
max,1115.000000,7.000000,38037.000000,7388.000000,1.000000,1.000000,75860.000000


In [0]:
# Determine if any there are any missing or null values.
df_train.isnull().sum()

Store                     0
DayOfWeek                 0
Date                      0
Sales                     0
Customers                 0
Open                      0
Promo                     0
StoreType                 0
Assortment                0
CompetitionDistance    1303
dtype: int64

In [0]:
# Drop (remove) rows that have missing values.
df_train.dropna(inplace=True)
df_train.isnull().sum()

Store                  0
DayOfWeek              0
Date                   0
Sales                  0
Customers              0
Open                   0
Promo                  0
StoreType              0
Assortment             0
CompetitionDistance    0
dtype: int64

### BASIC BUSINESS ANALYSIS

Next, we plot the data and seek to answer a few business questions about sales at Rossman stores.

In [0]:
# Review our distribution of daily sales.
df_train['Sales'].plot.hist(bins=20);

In [0]:
# Review our distribution of daily customers.
df_train['Customers'].plot.hist(bins=20);

In [0]:
# Generate a plot to show the relationship between customers and sales.
sns.scatterplot(x='Customers',y='Sales',data=df_train);

In [0]:
# Use a boxplot to review sales by store type.
sns.boxplot(x='StoreType', y='Sales', data=df_train);

In [0]:
# Use a boxplot to review sales by product assortment.
sns.boxplot(x='Assortment', y='Sales', data=df_train);

In [0]:
# Use a boxplot to review sales by promotion status.
sns.boxplot(x='Promo', y='Sales', data=df_train);

In [0]:
# Use a violinplot to review sales by promotion status.
sns.violinplot(x='Promo', y='Sales', data=df_train);

In [0]:
# Create a new data frame that only include days when stores are open.
df_train_open = df_train[df_train['Open'] > 0]
df_train_open.head(10)

In [0]:
# Re-run the boxplot to show sales by promotion status using filtered data.
sns.boxplot(x='Promo', y='Sales', data=df_train_open);

In [0]:
# Use a boxplot to show sales by store type and promotion.
sns.catplot("StoreType", "Sales", "Promo",
               data=df_train_open, kind="box");

### MODEL BUILDING (Linear Regression)

Finally, we will build a linear regression model to predict sales at various Rossman stores.

In [0]:
# Show the correlation between variables.
df_train.corr()

In [0]:
# Sepearate the dependent variable (y) from the independent variables (X).
X = df_train[['Open', 'Promo', 'CompetitionDistance']]
y = df_train['Sales']

In [0]:
# Pull out some of the training data (25%) and create a test dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [0]:
# Train the prediction model using the training data.
model = LinearRegression()
model.fit(X_train, y_train)

In [0]:
# Display the y-intercept and coefficients of our regression model.
# y = b + mx
print(model.intercept_)
print(model.coef_)

In [0]:
# Predict sales for a specific scenario.
new_X = [[1, 1, 1000]] # (open, promo, competition distance)
model.predict(new_X)

In [0]:
# Make predictions in the test data.
y_pred = model.predict(X_test)

In [0]:
# Evaluate the performance of the model (r-squared).
metrics.r2_score(y_test, y_pred)

In [0]:
# Compare the actual vs predicted value in the test data.
df_compare = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_compare.head(10)